In [13]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten 
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
import cv2
import numpy as np
import os
import pandas as pd

In [14]:
from keras.applications import VGG16
import ssl

# Workaround to avoid SSL certificate verification error
ssl._create_default_https_context = ssl._create_unverified_context

In [15]:
# VGG model requires 224*224 input so we are going to re-size all images
IMAGE_SIZE = [224, 224]

In [16]:
#file_path="../TEST/"

In [17]:
# example of progressively loading images from file
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# create generator
datagen = ImageDataGenerator(rescale = 1./255,
                             shear_range = 0.2,
                             zoom_range = 0.2,
                             horizontal_flip = True,
                             vertical_flip=True,
                             rotation_range=30)
# prepare an iterators for each dataset
train_it = datagen.flow_from_directory('./TRAIN.1', 
                                       class_mode='categorical',
                                       target_size=(224, 224),
                                       batch_size=2,
                                       seed=7)

test_it = datagen.flow_from_directory('./TEST',  
                                       class_mode='categorical',
                                       target_size=(224, 224),
                                       batch_size=2,
                                       seed=7)

Found 59922 images belonging to 2 classes.
Found 16726 images belonging to 2 classes.


In [18]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras import Sequential


## Using VGG16 model

In [19]:
# VGG16 model

In [20]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 31s 1us/step


In [21]:
for layer in vgg.layers:
  layer.trainable = False

In [22]:
output_classes = 2

In [23]:
##Adding flatten and Dense Layer
x = Flatten()(vgg.output)
x = Dense(1000, activation='relu')(x)

prediction = Dense(output_classes, activation='softmax')(x)

In [24]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [25]:
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [26]:
# tell the model what cost and optimization method to use
from tensorflow.keras.losses import CategoricalCrossentropy
model.compile(
  loss=CategoricalCrossentropy(),
  optimizer='adam',
  metrics=['accuracy']
)

In [29]:
history_vgg= model.fit(
  train_it,
  validation_data=test_it,
  epochs=1 # keep epochs=5 if you want to just check as it take more computational time
)

29961/29961 [==============================] - 2301s 77ms/step - loss: 3.6827 - accuracy: 0.8930 - val_loss: 5.3279 - val_accuracy: 0.8492


In [ ]:
##SAVING A MODEL

In [30]:
model.save("bio.h5")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


##If model is trained already,Then  Load model

In [31]:
from keras.models import load_model
 
# load model
model = load_model('bio.h5')

## Checking model with external image

In [35]:

from keras.preprocessing.image import load_img
image = load_img('./tomato.jpeg', target_size=(224, 224))
img = np.array(image)
img = img / 255.0
img = img.reshape(1,224,224,3)
label = model.predict(img)

1/1 [==============================] - 0s 269ms/step


In [36]:
l={"Biodegradable":label[0][0],"NON-Biodegradable":label[0][1]}
def get_key(val):
    for key, value in l.items():
         if val == value:
             return key
 
    return "key doesn't exist"
#label[0][1]

In [37]:
check=label.max()
get_key(check)

'Biodegradable'